In [1]:
import pandas as pd
import numpy as np
import os
import sys

from datetime import datetime
# Make `src` dir can be imported
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # /aicup-predict-energy-generation
sys.path.append(project_root_path)


from src.utils import choose_device

import matplotlib.pyplot as plt
import seaborn as sns

from src.fe_tools import create_time_features


In [2]:
df_raw_data = pd.read_csv('../data/processed_data/combined_data.csv')
df_raw_data['datetime'] = pd.to_datetime(df_raw_data['datetime'])
# make sure the sorting is correct
df_raw_data = df_raw_data.sort_values(by=['device','datetime']).reset_index(drop=True)

df_device = choose_device(df_raw_data, 'L10')
df_device = create_time_features(df_device)
df_device = df_device.reset_index(drop=True)

In [3]:
df_device

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30


In [4]:
df_device['group'] = df_device.index // 5
df_test = df_device.head(10).copy()
df_test

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group
0,2024-03-01 17:14:06,0.00,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0
1,2024-03-01 17:14:47,0.00,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0
2,2024-03-01 17:15:47,0.00,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0
3,2024-03-01 17:16:47,0.00,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,0
4,2024-03-01 17:17:47,0.00,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,0
5,2024-03-01 17:18:47,0.00,1017.53,15.75,93.38,582.50,0.10,L10,2024-03-01,3,1,17,18,9,1
6,2024-03-01 17:19:47,0.87,1017.52,15.67,92.41,502.50,0.07,L10,2024-03-01,3,1,17,19,9,1
7,2024-03-01 17:20:47,0.00,1017.55,15.65,92.90,459.17,0.06,L10,2024-03-01,3,1,17,20,9,1
8,2024-03-01 17:21:47,0.00,1017.51,15.69,93.62,473.33,0.07,L10,2024-03-01,3,1,17,21,9,1
9,2024-03-01 17:22:47,0.00,1017.48,15.72,93.93,475.00,0.06,L10,2024-03-01,3,1,17,22,9,1


In [5]:
df_test.loc[:, 'avg_sunlight'] = df_test.groupby(['group'])['sunlight'].transform('mean')
df_test['residual_sunlight'] = df_test['sunlight'] - df_test['avg_sunlight']
df_test

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group,avg_sunlight,residual_sunlight
0,2024-03-01 17:14:06,0.00,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0,696.916,-43.996
1,2024-03-01 17:14:47,0.00,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0,696.916,-14.416
2,2024-03-01 17:15:47,0.00,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0,696.916,53.084
3,2024-03-01 17:16:47,0.00,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,0,696.916,41.414
4,2024-03-01 17:17:47,0.00,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,0,696.916,-36.086
5,2024-03-01 17:18:47,0.00,1017.53,15.75,93.38,582.50,0.10,L10,2024-03-01,3,1,17,18,9,1,498.500,84.000
6,2024-03-01 17:19:47,0.87,1017.52,15.67,92.41,502.50,0.07,L10,2024-03-01,3,1,17,19,9,1,498.500,4.000
7,2024-03-01 17:20:47,0.00,1017.55,15.65,92.90,459.17,0.06,L10,2024-03-01,3,1,17,20,9,1,498.500,-39.330
8,2024-03-01 17:21:47,0.00,1017.51,15.69,93.62,473.33,0.07,L10,2024-03-01,3,1,17,21,9,1,498.500,-25.170
9,2024-03-01 17:22:47,0.00,1017.48,15.72,93.93,475.00,0.06,L10,2024-03-01,3,1,17,22,9,1,498.500,-23.500


In [15]:
df_test['rolling_avg_sunlight'] = df_test['sunlight'].rolling(window=2).mean().fillna(df_test['sunlight'])
df_test


,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group,avg_sunlight,residual_sunlight,rolling_avg_sunlight
0,2024-03-01 17:14:06,0.00,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0,696.916,-43.996,652.920
1,2024-03-01 17:14:47,0.00,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0,696.916,-14.416,667.710
2,2024-03-01 17:15:47,0.00,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0,696.916,53.084,716.250
3,2024-03-01 17:16:47,0.00,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,0,696.916,41.414,744.165
4,2024-03-01 17:17:47,0.00,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,0,696.916,-36.086,699.580
5,2024-03-01 17:18:47,0.00,1017.53,15.75,93.38,582.50,0.10,L10,2024-03-01,3,1,17,18,9,1,498.500,84.000,621.665
6,2024-03-01 17:19:47,0.87,1017.52,15.67,92.41,502.50,0.07,L10,2024-03-01,3,1,17,19,9,1,498.500,4.000,542.500
7,2024-03-01 17:20:47,0.00,1017.55,15.65,92.90,459.17,0.06,L10,2024-03-01,3,1,17,20,9,1,498.500,-39.330,480.835
8,2024-03-01 17:21:47,0.00,1017.51,15.69,93.62,473.33,0.07,L10,2024-03-01,3,1,17,21,9,1,498.500,-25.170,466.250
9,2024-03-01 17:22:47,0.00,1017.48,15.72,93.93,475.00,0.06,L10,2024-03-01,3,1,17,22,9,1,498.500,-23.500,474.165
